In [1]:
%load_ext autoreload
%autoreload 2
import telepot
bot = telepot.Bot('436447719:AAGq_iK2hE3cPFhmL9Jh53g9HyVkw3SXZbg')
def qq(txt):
    bot.sendMessage(1114926, txt)

In [2]:
qq('start')

In [3]:
import json
import gzip
from itertools import islice
from collections import Counter, defaultdict
from operator import itemgetter
import pyarrow.parquet as pq
import pyarrow as pa

import pandas as pd
from tqdm.notebook import tqdm
import dill
import os
from pqdm.processes import pqdm
import re

from workers import *
    
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models.callbacks import CallbackAny2Vec

import random
import collections
import gc

In [4]:
df = pd.read_pickle("df.pkl")

In [5]:
df.columns

Index(['vacancy_id', 'name', 'description', 'area_id', 'creation_date',
       'work_schedule', 'work_experience', 'compensation_from',
       'compensation_to', 'currency', 'key_skills', 'employer',
       'specializations', 'is_test', 'part', 'employment', 'spec_len',
       'groups_of_spec', 'len_groups', 'html_tags'],
      dtype='object')

In [6]:
df_test = df[df.is_test == False].copy()
del df

In [7]:
gc.collect()

20

In [8]:
%%time
df_test['clear_text'] = df_test.description.apply(remove_html)

CPU times: user 35.4 s, sys: 18.2 ms, total: 35.4 s
Wall time: 35.4 s


In [9]:
%%time
if __name__ ==  '__main__': 
    df_test['processed_text'] = parallelize_on_rows(df_test.clear_text, preprocess_text) 

CPU times: user 17.1 s, sys: 16.9 s, total: 34 s
Wall time: 18min 26s


In [5]:
# %%time
# df_test['processed_text'] = df_test.clear_text.apply(preprocess_text) 

In [10]:
df_text = df_test[['vacancy_id', 'name', 'specializations', 'key_skills', 'employer', 'groups_of_spec', 'processed_text']].copy()
del df_test
gc.collect()

20

In [11]:
df_text['key_skills'] = df_text['key_skills'].apply(' '.join)

In [12]:
%%time
if __name__ ==  '__main__':
    df_text['key_skills'] = parallelize_on_rows(df_text.key_skills, preprocess_text)

CPU times: user 1.49 s, sys: 5.02 s, total: 6.52 s
Wall time: 39.5 s


In [13]:
%%time
if __name__ ==  '__main__':
    df_text['name'] = parallelize_on_rows(df_text.name, preprocess_text)

CPU times: user 1.61 s, sys: 4.2 s, total: 5.81 s
Wall time: 36.5 s


In [14]:
%%time
df_text['key_list'] = df_text['key_skills'].apply(lambda x: x.split(' '))
df_text['name_list'] = df_text['name'].apply(lambda x: x.split(' '))

CPU times: user 3.67 s, sys: 661 ms, total: 4.33 s
Wall time: 4.33 s


In [15]:
qq('done')

In [16]:
df_text.head()

,vacancy_id,name,specializations,key_skills,employer,groups_of_spec,processed_text,key_list,name_list
4,2700005,разработчик python django,"[137, 221, 296]",,845fe6266ab1,[1],небольшой дружный команда требоваться опытный ...,[],"[разработчик, python, django]"
5,2700006,торговый представитель,"[149, 242]",работа текущий база клиент мерчендайзинг работ...,23ead4a8f164,[17],обязанность компания дистрибьютор продукт пита...,"[работа, текущий, база, клиент, мерчендайзинг,...","[торговый, представитель]"
6,2700007,менеджер оптовый продажа ткань,"[149, 196, 242, 324, 538]",b2b продажа активный продажа развитие продажа ...,9e3008e9f553,[17],стартекс крупный текстильный компания лидер ры...,"[b2b, продажа, активный, продажа, развитие, пр...","[менеджер, оптовый, продажа, ткань]"
8,2700010,врач клинический лабораторный диагностика бакт...,"[398, 537, 587]",,49f3280f53b6,[13],компания kdl клинико диагностический лаборатор...,[],"[врач, клинический, лабораторный, диагностика,..."
12,2700015,швея портной,"[129, 157, 516]",,24e6806caa88,"[18, 11, 29]",требоваться швея лаборант навык закройщик прои...,[],"[швея, портной]"


In [17]:
df_text.to_parquet('train_text.pq')

In [10]:
%%time
df_text['text_list'] = df_text['processed_text'].apply(lambda x: x.split(' '))

CPU times: user 17.2 s, sys: 5.74 s, total: 23 s
Wall time: 23 s


In [16]:
df_text.drop(columns = ['text_list'], inplace = True)

In [11]:
model = Doc2Vec.load('models/vector_size:300_min_count:2_epochs:10_window:7_seed:42_workers:4_negative:50.bin')

In [12]:
def inf_vec(txt):
    return model.infer_vector(txt.split(' '))

In [13]:
df_text['text_vector'] = pqdm(df_text['processed_text'], inf_vec, n_jobs=10)

In [18]:
df_text.head()

,vacancy_id,name,specializations,key_skills,employer,groups_of_spec,processed_text,key_list,name_list,text_vector
0,2700001,прораб великий устюг,None,,72dc8d78bbec,None,обязанность взаимодействие субподрядчик предос...,[],"[прораб, великий, устюг]","[-0.050694242, 0.4064839, 0.55667454, -0.16124..."
1,2700002,лаборант электротехнический лаборатория,None,office пользователь деловой общение деловой пе...,ba6dcbd0bcd1,None,связь расширение компания требоваться quot лаб...,"[office, пользователь, деловой, общение, делов...","[лаборант, электротехнический, лаборатория]","[-0.19709699, -0.35366777, 0.10756227, 0.23191..."
2,2700003,продавец консультант радуга парк,None,грамотный речь коммуникабельность активный про...,ddc67b017661,None,обязанность консультирование клиент торговый з...,"[грамотный, речь, коммуникабельность, активный...","[продавец, консультант, радуга, парк]","[-0.19167374, 0.19502771, 0.16828729, -0.04970..."
3,2700004,тестировщик команда saving products,None,,a621c7f876ec,None,команда saving products обеспечивать оформлени...,[],"[тестировщик, команда, saving, products]","[-0.7373982, -0.07595682, 1.0881921, -0.209517..."
7,2700009,сварщик аргонщик ручной дуговой,None,сварочный работа,d4e25650ee12,None,обязанность работа нержавейка толщина стенка д...,"[сварочный, работа]","[сварщик, аргонщик, ручной, дуговой]","[-0.27458248, -0.07573847, 0.30837265, 0.08945..."


In [17]:
df_text.to_parquet('df_vector_test.pq')

In [4]:
df_text = pd.read_parquet('df_vector_test.pq')

In [ ]:
%%time 
if __name__ ==  '__main__': 
    df_text['similar'] = parallelize_on_rows(df_text.text_vector, get_similar)

In [ ]:
# %%time
# df_text['similar']=df_text['text_vector'].apply(get_similar)

In [ ]:
qq('similarity - done')

In [25]:
vac_id = [id for id, sim in res]

In [26]:
df_tarin = pd.read_parquet('df_tarin.pq')

In [46]:
print(df_text.iloc[0].processed_text)

обязанность взаимодействие субподрядчик предоставление фронт работа контроль качество смр срок соответствие утверждать график контроль качество материал использовать работа подрядчик учет выполнять работа субподрядный организация участие подготовка перечень замечание выполнять работа субподрядный организация контроль исполнение замечание дефектный ведомость сбор подрядчик исполнительный документация составление план работа субподрядный организация выявление фактор препятствовать выполнение работа информирование руководитель контроль срок выполнение работа прямой подрядчик застройщик совмещать график работа планирование работа стэп своевременный выявление потребность ресурс участок формирование заявка ресурс выполнение работа сила стэп учет ресурс предоставление первичный отчетность выполнение работа сила стэп контроль исполнение правило инструктаж вверять рабочий начало работа осмотр место производство работа безопасность подтверждение объем работа выполнять субподрядчик подготовка баз

In [51]:
vac_id[0:4]

[1295984, 690584, 2632408, 40368]

In [52]:
print(df_tarin[df_tarin.vacancy_id == 40368].clear_text.values)

['Обязанности:  взаимодействие с субподрядчиками, предоставление им фронта работ, контроль качества СМР и сроков в соответствии с утвержденным графиком;  — контроль качества материалов, используемых в работе подрядчиками; — учет выполненных работ субподрядными организациями; — участие в подготовке перечня замечаний по выполненным работам субподрядными организациями; — контроль исполнения замечаний по дефектным ведомостям; — сбор от подрядчиков исполнительной документации; — составление плана работ субподрядных организаций; — выявление факторов препятствующих выполнению работ, информирование руководителя; — контроль сроков выполнения работ прямых подрядчиков Застройщика при совмещенном графике работ; — планирование работ СТЭП и своевременное выявление потребности в ресурсах на участке; — формирование заявок на ресурсы при выполнении работ силами СТЭП; — учет ресурсов и предоставление первичной отчетности при выполнении работ силами СТЭП; — контроль исполнения правил ОТ и ТБ; — инструкта